In [ ]:
!pip install torch==2.7.0 torchvision==0.22.0 torchaudio==2.7.0 --index-url https://download.pytorch.org/whl/cu126
!pip install transformers accelerate huggingface_hub pandas optuna pyproj einops geopy matplotlib

Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.0/867.0 MB 213.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 275.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 274.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 292.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 148.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 247.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 197.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 192.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━

In [20]:
!nvidia-smi

Fri May 23 08:01:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 570.133.07     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        On  |   00000000:C1:00.0 Off |                  N/A |
|  0%   57C    P8             19W /  350W |       0MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!curl -L -o data/im2gps3k/im2gps3k_places365.csv \
     https://raw.githubusercontent.com/TIBHannover/GeoEstimation/original_tf/meta/im2gps3k_places365.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  425k  100  425k    0     0   813k      0 --:--:-- --:--:-- --:--:--  813k


In [ ]:
from huggingface_hub import hf_hub_download, login

login(token="API_KEY")


# Specify the repository
repo_id = "Jia-py/MP16-Pro"

# List of files to download
files = [
    "metadata/MP16_Pro_filtered.csv",
    "metadata/MP16_Pro_places365.csv",
    "metadata/mp16_urls.csv",
    "mp-16-images00"
]

# Download each file
for file in files:
    try:
        file_path = hf_hub_download(repo_id=repo_id, filename=file, repo_type="dataset", local_dir="/data/mp16/")
        print(f"Downloaded {file} to {file_path}")
    except Exception as e:
        print(f"Error downloading {file}: {e}")



MP16_Pro_filtered.csv:   0%|          | 0.00/747M [00:00<?, ?B/s]

Downloaded metadata/MP16_Pro_filtered.csv to /data/mp16/metadata/MP16_Pro_filtered.csv


MP16_Pro_places365.csv:   0%|          | 0.00/859M [00:00<?, ?B/s]

Downloaded metadata/MP16_Pro_places365.csv to /data/mp16/metadata/MP16_Pro_places365.csv


mp16_urls.csv:   0%|          | 0.00/385M [00:00<?, ?B/s]

Downloaded metadata/mp16_urls.csv to /data/mp16/metadata/mp16_urls.csv


mp-16-images00:   0%|          | 0.00/21.5G [00:00<?, ?B/s]

Downloaded mp-16-images00 to /data/mp16/mp-16-images00


In [30]:
!mv /data/mp16/metadata/*.csv /data/mp16

In [7]:
!tar -xf /data/mp16/mp-16-images00 -C /data/mp16/

!cd /data/mp16/images && \
 ls > /data/mp16/all_files.txt && \
 total=$(wc -l < /data/mp16/all_files.txt) && \
 part=$((total / 10)) && \
 head -n "$part" /data/mp16/all_files.txt > /data/mp16/part_files.txt && \
 tar -cf /data/mp16/mp-16-images.tar -T /data/mp16/part_files.txt

tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [8]:
!ls /data/mp16/

all_files.txt  metadata		 mp-16-images00
images	       mp-16-images.tar  part_files.txt


In [ ]:
!python tune.py

[I 2025-05-23 08:14:30,620] Using an existing study with name 'mp16-sh-siren' instead of creating a new one.
read text data success
no exist tar index success, need building...
25642it [00:00, 29226.87it/s]
tar index buidling success
data columns:  22396
location from str to float success
logit_scale1 torch.Size([])
logit_scale2 torch.Size([])
logit_scale3 torch.Size([])
location_encoder.LocEnc0.capsule.1.weight torch.Size([1024, 512])
location_encoder.LocEnc0.capsule.1.bias torch.Size([1024])
location_encoder.LocEnc0.capsule.3.weight torch.Size([1024, 1024])
location_encoder.LocEnc0.capsule.3.bias torch.Size([1024])
location_encoder.LocEnc0.capsule.5.weight torch.Size([1024, 1024])
location_encoder.LocEnc0.capsule.5.bias torch.Size([1024])
location_encoder.LocEnc0.head.0.weight torch.Size([512, 1024])
location_encoder.LocEnc0.head.0.bias torch.Size([512])
location_encoder.LocEnc1.capsule.1.weight torch.Size([1024, 512])
location_encoder.LocEnc1.capsule.1.bias torch.Size([1024])
locati